In [64]:
import pandas as pd
from pandas import DataFrame
from pandas import concat
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
import pandas as pd
import numpy as np
import random
from sklearn.cross_validation import cross_val_score # K折交叉验证模块
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing
from sklearn import svm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFE
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from sklearn.preprocessing import Normalizer
from imblearn.over_sampling import SMOTE, ADASYN
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.multiclass import OutputCodeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import f_regression
from sklearn.feature_selection import f_classif
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import ExtraTreesClassifier
from imblearn.pipeline import Pipeline as ImPineline
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import IsolationForest
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.grid_search import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from scipy.signal import lfilter
import pywt

import time
%matplotlib inline

In [65]:
# spec = total_train_feature.iloc[:,:-1].head()

def specLineNum(spec, windowSize=2, threshold=0.8):
    fs = pd.Series()
    ss = pd.Series()
    for i in range(windowSize, spec.shape[1]):
        s = spec.iloc[:,i-windowSize]
        e = spec.iloc[:,i]
        isFs = (e>threshold) & ((e-s)>threshold)
        isSS = (e<threshold) & ((e-s)<-threshold)
        if(fs.size==0):
            fs = isFs.astype(int)
        else:
            fs = fs + isFs.astype(int)
        if(ss.size==0):
            ss = isSS.astype(int)
        else:
            ss = ss + isSS.astype(int)
    combine = pd.concat([fs, ss], axis=1)
    combine.columns = ['fs','ss']
    return combine

In [66]:
def waveletFilter(x, wavelet="db4", level=1, mode="hard", scaleNum=0.5):
    coeff = pywt.wavedec( x, wavelet, mode="per", axis=1 )
    uthresh = scaleNum * np.sqrt( 2*np.log( x.shape[1] ) )
    coeff[1:] = ( pywt.threshold( i, value=uthresh, mode=mode ) for i in coeff[1:] )
    coeff
    # # reconstruct the signal using the thresholded coefficients
    y = pywt.waverec( coeff, wavelet, mode="per" )
    y = pd.DataFrame(y)
    return y

In [67]:
def qsoFeature(data, doScale=True):
    if doScale:
        data = pd.DataFrame(scale(data, axis=1))
    std1 = data.iloc[:,850:880].std(axis=1)
    std2 = data.iloc[:,885:898].std(axis=1)
    std3 = data.iloc[:,900:1000].std(axis=1)
    f1 = (std1<0.3) & (std2>0.35) & (std3<0.35)
    f3 = (std2*std2)/(std1*std3+0.1)
    d1 = std1+0.2
    f2 = (std2>d1)
    qsof = pd.concat([f1, f2, f3], axis=1)
    qsof.columns = ['qso_f1','qso_f2','qso_f3']
    return (qsof)

In [68]:
def staticFeature(data, doScale=False):
    if doScale:
        data = pd.DataFrame(scale(data, axis=1))
    mean =data.mean(axis=1)
    std =data.std(axis=1)
    d_max =data.max(axis=1)
    d_min =data.min(axis=1)
    d_25 =data.quantile(0.25, axis=1)
    d_50 =data.quantile(0.5, axis=1)
    d_75 =data.quantile(0.75, axis=1)
    return pd.concat([mean, std, d_max, d_min, d_25, d_50, d_75], axis=1)

In [69]:
def intervalEWFeature(d, start=0, end=-1, filterThrehold=1):
    cp = d
#     cp[abs(cp)<filterThrehold] = 0
    rd = cp.iloc[:, start:end]
    specLines = specLineNum(rd[abs(rd)>filterThrehold])
    ew_absorb = rd[(rd<0)&(abs(rd)>filterThrehold)].sum(axis=1)
    ew_fs = rd[(rd>0)&(abs(rd)>filterThrehold)].sum(axis=1)
    rd_mean = rd.mean(axis=1)+0.1
#     st_feature = staticFeature(d)
#     diff_feature = extractFeature(cp.diff(axis=1).iloc[:,1:], 3, levelFeature=False, smooth=False)

    return pd.concat([ew_absorb, ew_fs, specLines], axis=1)

def smooth(filterSet):
    n = 5  # the larger n is, the smoother curve will be
    b = [1.0 / n] * n
    a = 1
    filterSet = lfilter(b, a, filterSet, axis=1)
    return pd.DataFrame(filterSet)

def EWByRangeList(d, rangeList, filterThrehold=1, isSmooth=False, doScale=True):
    if doScale:
        d = pd.DataFrame(scale(d, axis=1))
    if isSmooth:
        d=smooth(d)
    ewfAll = pd.DataFrame()
    for r in rangeList:
        start, end = r
        ewf = intervalEWFeature(d, start, end, filterThrehold)
        newNames = ['ew_' + str(start) + '_' + str(end) + '_' + str(v) for v in ewf.columns.values]
        ewf.columns = newNames
        ewfAll = pd.concat([ewfAll,ewf], axis=1)
    return ewfAll

In [70]:
def submit(fileName, testTotal, preds):
    submission = testTotal[['id']]
    submission['pred'] = preds
    submission.to_csv('./predicts/'+fileName, header=False, index=False)
    return submission
    

In [71]:
def extractFeature(train_feature, partition=2, randomPartion=False, prefix="p", onlyAvg=False, filterLow=False, levelFeature=False, scaleSpec=False, smooth=False):
    if scaleSpec:
        train_feature = pd.DataFrame(scale(train_feature, axis=1))
    if smooth:
        train_feature = train_feature.rolling(100, axis=1).mean()
    if filterLow:
        train_feature[train_feature<1] = 0
    plen = train_feature.shape[1]/partition
    features = pd.DataFrame()
    avgs = pd.DataFrame()
    for i in range(0, partition):
        pstart = random.randint(0, train_feature.shape[1]-plen-1)
        pendExclue = pstart + plen + 1
        avgC = train_feature.iloc[:, pstart:pendExclue].mean(axis=1)
        stdC = train_feature.iloc[:, pstart:pendExclue].std(axis=1)
        maxC = train_feature.iloc[:, pstart:pendExclue].max(axis=1)
        minC = train_feature.iloc[:, pstart:pendExclue].min(axis=1)
        medianC = train_feature.iloc[:, pstart:pendExclue].median(axis=1)
        diffC = train_feature.iloc[:, pstart:pendExclue].diff(axis=1).iloc[:,1:]
        features[prefix+'_avg'+str(i)] = avgC
        avgs[prefix+'_avg'+str(i)] = avgC
        if onlyAvg==False:
            features[prefix+'_std'+str(i)] = stdC
            features[prefix+'_median'+str(i)] = medianC
            features[prefix+'_max'+str(i)] = maxC
            features[prefix+'_min'+str(i)] = minC
            features[prefix+'_avg_m_std'+str(i)] = stdC/avgC
            features[prefix+'_max_m_avg'+str(i)] = maxC/avgC
            features[prefix+'_min_m_avg'+str(i)] = minC/avgC
            features[prefix+'_media_m_avg'+str(i)] = medianC/avgC
    if levelFeature:
        level_feature = extractFeature(avgs, prefix="l", partition = 4)
        features = pd.concat([features, level_feature], axis = 1)
    return features.fillna(0)

In [75]:
def readTotalFeature(fileName, maxFileIndex, hasLabel=True, partition = 20, ewFeature=True, addQsoFeature=True, doWavlet=False,diffFeature=True, addFFt=True):
    total_train_feature = pd.DataFrame()
    for i in range(0, maxFileIndex+1):
        print('processing:' + str(i))
        train_set = pd.read_csv(fileName + str(i) +'.csv', header=None)
        renameCol = {0:'id'}
        if hasLabel:
            renameCol[train_set.shape[1]-1] = 'label'
        train_set.rename(columns=renameCol, inplace=True)
        if hasLabel:
            processed_train = pd.DataFrame()
            rawFeature = train_set.iloc[:,1:-1]
        else:
            rawFeature = train_set.iloc[:,1:]
        if doWavlet:
            rawFeature = waveletFilter(rawFeature, scaleNum=1)
        processed_train = extractFeature(rawFeature, partition, levelFeature=False, smooth=False)
        if ewFeature:
            rangeList = [(0,20),(30,100),(1040,1060),(1380,1394),(2344,2360),(2360,2380),(2420,2440),(0,300), (0, 500), (500, 1000),(850, 1000), (1000, 1300),(1500,2000), (1500, 2600), (2500, 2600)]
            ew_feature = EWByRangeList(rawFeature, rangeList, filterThrehold=0.5, doScale=True)
            processed_train = pd.concat([processed_train, ew_feature], axis=1)
        if addQsoFeature:
            qso_feature = qsoFeature(rawFeature)
            processed_train = pd.concat([processed_train, qso_feature], axis=1)
        if addFFt:
            ff =abs(pd.DataFrame(np.fft.fft(rawFeature, axis=1))).iloc[:, :(rawFeature.shape[1]/2)]
            fftfeature = extractFeature(ff, 20, levelFeature=False, smooth=False, scaleSpec=True)
            processed_train = pd.concat([processed_train, fftfeature], axis=1)
        st_feature = staticFeature(rawFeature)            
        processed_train = pd.concat([processed_train, st_feature], axis=1)
        if hasLabel:
            processed_train['label'] = train_set.label
        else:
            processed_train['id'] = train_set.id
        total_train_feature = total_train_feature.append(processed_train, ignore_index=True)
    return total_train_feature.sample(total_train_feature.shape[0], replace=False)

In [76]:
validMode = True
reFeature = True
partition = 30
rng = np.random.RandomState(42)

In [77]:
if reFeature:
    total_train_feature = readTotalFeature('./train_feature_all_', 1, partition=partition)
    total_train_feature.to_csv('./train_feature_check_point.csv', index=False)
else :
    total_train_feature = pd.read_csv('./train_feature_check_point.csv')

processing:0
processing:1


In [78]:
total_train_feature = total_train_feature.fillna(0)

In [79]:
total_train_feature.head()

,p_avg0,p_std0,p_median0,p_max0,p_min0,p_avg_m_std0,p_max_m_avg0,p_min_m_avg0,p_media_m_avg0,p_avg1,...,p_min_m_avg19,p_media_m_avg19,0.0,1.0,2.0,3.0,0.25,0.5,0.75,label
656,786.785816,39.369763,794.333217,841.916935,588.718892,0.050039,1.070071,0.748258,1.009593,906.890322,...,1.427798,1.022068,1134.099357,250.983930,1627.595339,135.017161,930.808191,1164.534470,1348.672034,star
3461,165.447738,19.859533,167.809266,210.146937,112.183132,0.120035,1.270171,0.678058,1.014274,255.123616,...,2.481079,1.242246,324.312665,250.556795,966.109459,-845.158522,199.656986,329.871127,533.636268,star
14572,379.621699,8.339597,380.797810,400.585734,345.672386,0.021968,1.055223,0.910571,1.003098,303.963992,...,1.373235,1.024532,364.959125,87.539642,554.369726,-14.321416,324.954403,379.624347,427.611897,star
13932,1625.221159,21.323329,1625.173732,1696.501912,1576.715936,0.013120,1.043859,0.970155,0.999971,2497.912594,...,1.181756,1.012524,2017.433449,494.732382,2911.783298,608.481375,1563.884172,2036.561567,2461.082799,star
9111,101.587052,5.043986,102.498268,111.308467,75.197421,0.049652,1.095695,0.740226,1.008970,117.272664,...,1.190901,0.996732,147.975272,31.870797,204.834113,28.534911,120.294575,151.798209,177.398418,star


In [80]:
# total_train_feature.iloc[:,1:-1].diff(axis=1).iloc[:,1:].head()

In [81]:
total_train_feature[total_train_feature.label=='unknown'].head(1000)

,p_avg0,p_std0,p_median0,p_max0,p_min0,p_avg_m_std0,p_max_m_avg0,p_min_m_avg0,p_media_m_avg0,p_avg1,...,p_min_m_avg19,p_media_m_avg19,0.0,1.0,2.0,3.0,0.25,0.5,0.75,label
4791,9.409646,4.022907,8.298014,25.994258,0.984585,0.427530,2.762512,0.104636,0.881862,7.315901,...,-5.195457,0.939307,11.817902,8.430520,96.578427,-72.782921,7.881030,9.821426,15.064022,unknown
5041,152.836355,9.889346,152.403148,177.213391,129.603972,0.064705,1.159498,0.847992,0.997166,183.185760,...,1.458096,1.001981,170.655551,120.170119,5007.873558,-40.653340,150.873590,166.254354,181.793939,unknown
14105,4.574419,1.380835,4.805509,8.569609,1.289574,0.301860,1.873376,0.281910,1.050518,7.086972,...,2.728762,1.038386,6.228087,4.071733,34.015935,-33.469684,4.084024,5.394633,7.468547,unknown
11629,37.248336,6.470390,37.386905,54.479622,20.038139,0.173710,1.462606,0.537961,1.003720,14.060675,...,4.824934,0.627626,20.668787,24.673065,87.646522,-247.081450,14.921202,26.829954,34.660180,unknown
16709,10.782185,2.899676,10.649413,23.164026,2.866889,0.268932,2.148361,0.265891,0.987686,5.425009,...,4.683738,1.026076,7.437261,6.378738,71.231793,-42.228951,4.892781,7.919466,10.576137,unknown
17233,272.882027,31.573500,269.570352,397.938352,223.003030,0.115704,1.458280,0.817214,0.987864,166.328855,...,-53.449100,-8.122003,242.264774,91.460945,1304.016487,-26.326704,195.650217,235.582859,267.377658,unknown
3508,55.018636,6.925643,55.172253,72.395204,34.827728,0.125878,1.315831,0.633017,1.002792,59.984871,...,5.334522,1.454915,28.896485,48.007316,96.978471,-339.146825,16.925422,51.381833,58.376484,unknown
12679,48.849530,6.635584,48.651094,71.798111,25.183741,0.135837,1.469781,0.515537,0.995938,68.675068,...,6.039966,0.897040,63.648473,32.149719,373.670640,-27.690414,43.796683,58.935800,75.446054,unknown
6554,137051.125761,16704.391245,136977.593297,165979.627451,67055.463846,0.121884,1.211078,0.489273,0.999463,193.894704,...,1.380134,1.014714,67759.708735,52189.349610,196729.244424,-52872.418130,12811.396336,70061.538269,116577.613164,unknown
10626,6.827367,1.420914,6.819259,11.317050,1.904604,0.208120,1.657601,0.278966,0.998812,8.698792,...,4.573412,1.414720,8.007825,3.803724,54.737048,-36.941508,6.464642,7.473142,9.276625,unknown


In [82]:
X = total_train_feature.iloc[:,:-1]
Y = total_train_feature.iloc[:,-1]

# Split the dataset in two equal parts
x, t_x, y, t_y = train_test_split(
    X, Y, test_size=0.3, random_state=0)
x.shape


(14000, 520)

In [83]:
smote = SMOTE()
anova_filter = SelectKBest(f_classif, k=(int)(x.shape[1]*0.9))
model = RandomForestClassifier(n_jobs=8, class_weight='balanced', max_depth=20,   verbose=1,random_state=rng)
# model = AdaBoostClassifier(random_state=rng)
# model = GradientBoostingClassifier(random_state=rng,verbose=1)
# model = XGBClassifier()
# model = KNeighborsClassifier(n_neighbors = 3, n_jobs=-1)
scaler = StandardScaler()
clf = ImPineline([
#     ('anova', anova_filter),
    ('scaler',scaler), 
    ('smote',smote),
    ('model', model)
])


In [84]:
if validMode:
    rs = clf.fit(x, y)
    scores = cross_val_score(clf, x, y, cv=3, n_jobs=-1, verbose=1, scoring='f1_macro')
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
#     preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
#     print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  

[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    2.0s remaining:    1.3s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    3.2s finished
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    3.4s remaining:    2.3s
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    3.4s remaining:    2.3s
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    3.3s remaining:    2.2s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    4.4s finished
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    4.7s finished
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    4.2s finished
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=8)]

Accuracy: 0.56 (+/- 0.06)


[Parallel(n_jobs=-1)]: Done   3 out of   3 | elapsed:    7.5s finished


In [85]:
if validMode:
    smote = SMOTE()
    parameters = {
        'max_depth':[20, 60, 70, 80],  #70
        'max_features':[0.5,0.6,0.7], #0.6
        'min_samples_split':[2, 4, 8, 20], #2
        'min_samples_leaf':[1,4,8,20], #1
        'min_weight_fraction_leaf':[0,0.05,0.1,0.2], #0
        'max_leaf_nodes':[800, 1000, 1200], #1000
         'bootstrap':[True, False], #False
    #     'oob_score':[True, False], #False
    #     'class_weight':[None,'balanced'], #balanced
        'n_estimators':[10, 20, 100], #10
    #     'criterion':['gini','entropy']
    }
    clf = RandomizedSearchCV(RandomForestClassifier(n_jobs=8, class_weight='balanced'), parameters, cv=3,
                           scoring='f1_macro', n_jobs=8, verbose=2)
    clf.fit(x, y)
    print(clf.best_params_)
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r" % (mean, std * 2, params))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] max_leaf_nodes=1200, bootstrap=False, min_samples_leaf=8, n_estimators=20, min_samples_split=2, min_weight_fraction_leaf=0.05, max_features=0.7, max_depth=80 
[CV] max_leaf_nodes=1200, bootstrap=False, min_samples_leaf=8, n_estimators=20, min_samples_split=2, min_weight_fraction_leaf=0.05, max_features=0.7, max_depth=80 
[CV] max_leaf_nodes=1200, bootstrap=False, min_samples_leaf=8, n_estimators=20, min_samples_split=2, min_weight_fraction_leaf=0.05, max_features=0.7, max_depth=80 
[CV] max_leaf_nodes=1200, bootstrap=True, min_samples_leaf=4, n_estimators=20, min_samples_split=2, min_weight_fraction_leaf=0.2, max_features=0.6, max_depth=70 
[CV] max_leaf_nodes=1200, bootstrap=True, min_samples_leaf=4, n_estimators=20, min_samples_split=2, min_weight_fraction_leaf=0.2, max_features=0.6, max_depth=70 
[CV] max_leaf_nodes=1200, bootstrap=True, min_samples_leaf=4, n_estimators=20, min_samples_split=2, min_weight_fraction_lea

Process PoolWorker-34:
Traceback (most recent call last):
  File "/Users/cql/anaconda2/lib/python2.7/multiprocessing/process.py", line 267, in _bootstrap
    self.run()
  File "/Users/cql/anaconda2/lib/python2.7/multiprocessing/process.py", line 114, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/cql/anaconda2/lib/python2.7/multiprocessing/pool.py", line 102, in worker
    task = get()
  File "/Users/cql/anaconda2/lib/python2.7/site-packages/sklearn/externals/joblib/pool.py", line 362, in get
    return recv()
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
if validMode:
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
#     preds_test = carbateQso(t_x, preds_test)
#     clf.score(t_x, t_y)
    print("%0.3f" % f1_score(t_y, preds_test, average='macro'))  
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [ ]:
def carbateQso(t_x, preds_test):
    fPreds = pd.concat([t_x.reset_index(), pd.DataFrame(preds_test)], axis=1)
    fPreds[(fPreds.iloc[:,-1]=='unknown')&(fPreds.qso_f3>2)].iloc[:,-1] = 'qso'
    return fPreds.iloc[:,-1].values

In [ ]:
if validMode:
    clf = RandomForestClassifier(n_jobs=-1, class_weight='balanced',  verbose=1 )
    rs = clf.fit(x, y)
    preds_test = clf.predict(t_x)
    # clf.score(t_x, t_y)
    print('f1: %.2f' % f1_score(t_y, preds_test, average='macro')) 
    print(classification_report(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))
    print(confusion_matrix(t_y, preds_test, labels=['galaxy', 'qso', 'star', 'unknown']))

In [ ]:
if validMode:
    importances = model.feature_importances_
    std = np.std([tree.feature_importances_ for tree in model.estimators_],
                 axis=0)
    indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    print("Feature ranking:")

    for f in range(X.shape[1]):
        print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))

    # Plot the feature importances of the forest
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", yerr=std[indices], align="center")
    plt.xticks(range(X.shape[1]), indices)
    plt.xlim([-1, X.shape[1]])
    plt.show()


In [271]:
if reFeature:
    testTotal = readTotalFeature('./test_feature_all_', 9, hasLabel=False, partition=partition)
    testTotal.to_csv('./test_feature_checkpoint.csv', index=False)
    testTotal.head()
else :
    testTotal = pd.read_csv('./test_feature_checkpoint.csv')

processing:0
processing:1
processing:2
processing:3
processing:4
processing:5
processing:6
processing:7
processing:8
processing:9


In [272]:
testTotal.shape

(100000, 334)

In [273]:
X.shape

(483851, 333)

In [274]:
clf.fit(X, Y)



[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:  1.6min remaining:  1.1min
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:  2.7min finished


Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('smote', SMOTE(k=None, k_neighbors=5, kind='regular', m=None, m_neighbors=10, n_jobs=1,
   out_step=0.5, random_state=None, ratio='auto', svm_estimator=None)), ('model', RandomForestClassifier(bootstrap=True, class_weight=...random_state=<mtrand.RandomState object at 0x1c1288caf0>,
            verbose=1, warm_start=False))])

In [275]:
test_x = testTotal.iloc[:,:-1]
pred = clf.predict(test_x)
timestr = time.strftime("%m%d_%H%M")
rs = submit(timestr+'_rf_ew_more_erange.csv',testTotal, pred)


[Parallel(n_jobs=8)]: Done   6 out of  10 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done  10 out of  10 | elapsed:    0.1s finished
/Users/cql/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [276]:
rs.pred.value_counts()

star       85831
unknown    10545
galaxy      2383
qso         1241
Name: pred, dtype: int64

In [ ]:
total_train_feature.head()

In [ ]:
total_train_feature[:,:-1]